In [74]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 


# sklearn preprocessing for dealing with categorical variables
import sklearn
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, mean_squared_error, accuracy_score,roc_auc_score,recall_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import KNNImputer
from sklearn.metrics import make_scorer

from tqdm import tqdm

import h2o
from h2o.estimators import H2ORandomForestEstimator


from xgboost import XGBClassifier

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')



# matplotlib and seaborn for plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import mlflow
import mlflow.sklearn




import gc
import time
import warnings

from contextlib import contextmanager
from lightgbm import LGBMClassifier

## Data Train

In [75]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    
    # Nombre de colonnes supprimées (colonnes catégorielles remplacées par les dummies)
    nb_columns_removed = len(categorical_columns)

    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns,nb_columns_removed

In [76]:
# Preprocess application_train.csv and application_test.csv
def application_train(num_rows = None, nan_as_category = False):
    # Read data and merge
    df = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\application_train.csv', nrows= num_rows)
    ##test_df = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\application_test.csv', nrows= num_rows)
    print("Train samples: {}".format(len(df)))
    print("taille du df initial: {}".format(df.shape)) 
    #df = pd.concat([df, test_df], ignore_index=True)
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    binary_cols = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']
    for bin_feature in binary_cols:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols,nb_suppr_cols = one_hot_encoder(df, nan_as_category=True)
    print("Nouvelles colonnes créés encoder: {}".format(len(cat_cols)))
    print("Anciennes colonnes supprimées: {}".format(nb_suppr_cols))

    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

    print("Nouvelles colonnes créés ratio: {}".format(5))

    # Identify non-categorical columns
    encoded_columns = binary_cols + cat_cols
    # Use set operations to exclude 'SK_ID_CURR' and 'TARGET'
    non_categorical_cols = df.columns.difference(set(encoded_columns).union({'SK_ID_CURR', 'TARGET'}))

    


        
    gc.collect()

    # sauvegarder les colonnes encodées
    encoded_columns = binary_cols + cat_cols
    
    return df, encoded_columns
    

In [77]:
df_data,col_encod=application_train(num_rows = None, nan_as_category = False)

Train samples: 307511
taille du df initial: (307511, 122)
Nouvelles colonnes créés encoder: 146
Anciennes colonnes supprimées: 13
Nouvelles colonnes créés ratio: 5


In [78]:
df_data.shape

(307507, 260)

In [79]:
df_data.head

<bound method NDFrame.head of         SK_ID_CURR  TARGET  CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  \
0           100002       1            0             0                0   
1           100003       0            1             0                1   
2           100004       0            0             1                0   
3           100006       0            1             0                0   
4           100007       0            0             0                0   
...            ...     ...          ...           ...              ...   
307506      456251       0            0             0                1   
307507      456252       0            1             0                0   
307508      456253       0            1             0                0   
307509      456254       1            1             0                0   
307510      456255       0            1             0                1   

        CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0             

## Data Bureau and Balance

In [80]:
import pandas as pd
import numpy as np
import gc
from sklearn.preprocessing import StandardScaler

# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows=None, nan_as_category=True):
    bureau = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\bureau.csv', nrows=num_rows)
    bb = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\bureau_balance.csv', nrows=num_rows)

    print("taille du df bureau initial: {}".format(bureau.shape))
    print("taille du df bb: {}".format(bb.shape))      
    
    # One-hot encoding for categorical variables
    bb, bb_cat,bb_suppr_cols = one_hot_encoder(bb, nan_as_category=True)
    bureau, bureau_cat,bureau_suppr_cols = one_hot_encoder(bureau, nan_as_category=True)
    print("Nouvelles colonnes bb créés encoder: {}".format(len(bb_cat)))
    print("Anciennes colonnes bb supprimées: {}".format(bb_suppr_cols))
    print("Nouvelles colonnes bureau créés encoder: {}".format(len(bureau_cat)))
    print("Anciennes colonnes bb supprimées: {}".format(bureau_suppr_cols))
    
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    
    # Merge aggregated bureau balance data
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)
    
    # Garbage collection
    del bb, bb_agg
    gc.collect()

    # Bureau and bureau_balance numeric features aggregations
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat:
        cat_aggregations[cat] = ['mean']
    for cat in bb_cat:
        cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

    # Identify non-categorical columns
    encoded_columns = bb_cat + bureau_cat
    # Exclure explicitement 'SK_ID_CURR' des colonnes à scaler
    non_categorical_cols = bureau_agg.columns.difference(encoded_columns).difference(['SK_ID_CURR'])

     
    
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')

    # Garbage collection
    del active, active_agg
    gc.collect()
    
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')

    # Après la jointure, réinitialiser l'index de bureau_agg pour récupérer 'SK_ID_CURR' comme colonne
    bureau_agg = bureau_agg.reset_index()

    # Cleanup
    del closed, closed_agg, bureau
    gc.collect()

    print("taille du df bureau_agg: {}".format(bureau_agg.shape))  
    
    return bureau_agg


In [81]:
df_bureau=bureau_and_balance(num_rows = None, nan_as_category = True)

taille du df bureau initial: (1716428, 17)
taille du df bb: (27299925, 3)
Nouvelles colonnes bb créés encoder: 9
Anciennes colonnes bb supprimées: 1
Nouvelles colonnes bureau créés encoder: 26
Anciennes colonnes bb supprimées: 3
taille du df bureau_agg: (305811, 117)


In [82]:
# Mettre 'SK_ID_CURR' en index de df_bureau
df_bureau.set_index('SK_ID_CURR', inplace=True)
# Faire la jointure
df_data = df_data.join(df_bureau, how='left', on='SK_ID_CURR')

In [83]:
df_data.shape

(307507, 376)

In [84]:
df_data.head

<bound method NDFrame.head of         SK_ID_CURR  TARGET  CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  \
0           100002       1            0             0                0   
1           100003       0            1             0                1   
2           100004       0            0             1                0   
3           100006       0            1             0                0   
4           100007       0            0             0                0   
...            ...     ...          ...           ...              ...   
307506      456251       0            0             0                1   
307507      456252       0            1             0                0   
307508      456253       0            1             0                0   
307509      456254       1            1             0                0   
307510      456255       0            1             0                1   

        CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0             

## Data Previous

In [85]:
# Preprocess previous_applications.csv
def previous_applications(num_rows=None, nan_as_category=True):
    prev = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\previous_application.csv', nrows=num_rows)
    prev, cat_cols,prev_suppr_cols = one_hot_encoder(prev, nan_as_category=nan_as_category)

    # Remplacer les valeurs 365.243 par NaN
    days_columns = [
        'DAYS_FIRST_DRAWING', 
        'DAYS_FIRST_DUE', 
        'DAYS_LAST_DUE_1ST_VERSION', 
        'DAYS_LAST_DUE', 
        'DAYS_TERMINATION'
    ]
    for col in days_columns:
        prev[col].replace(365243, np.nan, inplace=True)

    # Ajouter une nouvelle fonctionnalité : pourcentage de demande / valeur reçue
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']

    # Agrégations pour les caractéristiques numériques
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }

    # Agrégations pour les caractéristiques catégorielles
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']

    # Agrégation des données
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])

    # Traitement des valeurs infinies
    prev_agg.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Imputation des valeurs manquantes
    prev_agg.fillna(prev_agg.mean(), inplace=True)

    # Identifier les colonnes non catégorielles à mettre à l'échelle
    encoded_columns = pd.Index(cat_cols)  # Convertir en Index
    non_categorical_cols = prev_agg.columns.difference(encoded_columns).difference(['SK_ID_CURR'])  # Exclure SK_ID_CURR
    


    # Applications précédentes : applications approuvées - uniquement les caractéristiques numériques
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')

    # Applications précédentes : applications refusées - uniquement les caractéristiques numériques
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')

    # Nettoyage de la mémoire
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()

    return prev_agg


In [86]:
df_previous=previous_applications(num_rows = None, nan_as_category = True)

In [87]:
df_previous.info()

<class 'pandas.core.frame.DataFrame'>
Index: 338857 entries, 100001 to 456255
Columns: 249 entries, PREV_AMT_ANNUITY_MIN to REFUSED_CNT_PAYMENT_SUM
dtypes: float64(245), int64(4)
memory usage: 646.3 MB


In [88]:
df_data = df_data.join(df_previous, how='left', on='SK_ID_CURR')

In [89]:
df_data.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,REFUSED_HOUR_APPR_PROCESS_START_MAX,REFUSED_HOUR_APPR_PROCESS_START_MEAN,REFUSED_RATE_DOWN_PAYMENT_MIN,REFUSED_RATE_DOWN_PAYMENT_MAX,REFUSED_RATE_DOWN_PAYMENT_MEAN,REFUSED_DAYS_DECISION_MIN,REFUSED_DAYS_DECISION_MAX,REFUSED_DAYS_DECISION_MEAN,REFUSED_CNT_PAYMENT_MEAN,REFUSED_CNT_PAYMENT_SUM
0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,15.0,15.0,NaN,NaN,NaN,-181.0,-181.0,-181.0,48.0,48.0
4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307507 entries, 0 to 307510
Columns: 625 entries, SK_ID_CURR to REFUSED_CNT_PAYMENT_SUM
dtypes: bool(146), float64(436), int64(43)
memory usage: 1.1 GB


## Pos_Cash_Balance

In [91]:
# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows=None, nan_as_category=True):
    pos = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\POS_CASH_balance.csv', nrows=num_rows)
    pos, cat_cols,pos_suppr_cols = one_hot_encoder(pos, nan_as_category=nan_as_category)

    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    
    # Compter les comptes de POS Cash
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    
    # Traitement des valeurs infinies
    pos_agg.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Imputation des valeurs manquantes
    pos_agg.fillna(pos_agg.mean(), inplace=True)

   # Identify columns that should not be scaled (categorical + SK_ID_CURR)
    encoded_columns = cat_cols  # Assuming cat_cols contains all categorical columns
    non_categorical_cols = pos_agg.columns.difference(encoded_columns).difference(['SK_ID_CURR'])  # Exclure SK_ID_CURR

    
   
    # Nettoyage de la mémoire
    del pos
    gc.collect()

    return pos_agg

In [92]:
df_Pos_Cash=pos_cash(num_rows = None, nan_as_category = True)

In [93]:
df_Pos_Cash.info()

<class 'pandas.core.frame.DataFrame'>
Index: 337252 entries, 100001 to 456255
Data columns (total 18 columns):
 #   Column                                               Non-Null Count   Dtype  
---  ------                                               --------------   -----  
 0   POS_MONTHS_BALANCE_MAX                               337252 non-null  int64  
 1   POS_MONTHS_BALANCE_MEAN                              337252 non-null  float64
 2   POS_MONTHS_BALANCE_SIZE                              337252 non-null  int64  
 3   POS_SK_DPD_MAX                                       337252 non-null  int64  
 4   POS_SK_DPD_MEAN                                      337252 non-null  float64
 5   POS_SK_DPD_DEF_MAX                                   337252 non-null  int64  
 6   POS_SK_DPD_DEF_MEAN                                  337252 non-null  float64
 7   POS_NAME_CONTRACT_STATUS_Active_MEAN                 337252 non-null  float64
 8   POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN         3

In [94]:
df_Pos_Cash.head()

,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,
100001,-53,-72.555556,9,7,0.777778,7,0.777778,0.777778,0.0,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9
100002,-1,-10.000000,19,0,0.000000,0,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19
100003,-18,-43.785714,28,0,0.000000,0,0.000000,0.928571,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28
100004,-24,-25.500000,4,0,0.000000,0,0.000000,0.750000,0.0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4
100005,-15,-20.000000,11,0,0.000000,0,0.000000,0.818182,0.0,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11


In [95]:
df_data = df_data.join(df_Pos_Cash, how='left', on='SK_ID_CURR')

In [96]:
df_data.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT
0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0.0,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0
2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0.0,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0
3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0.0,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0
4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0.0,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0


In [97]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307507 entries, 0 to 307510
Columns: 643 entries, SK_ID_CURR to POS_COUNT
dtypes: bool(146), float64(454), int64(43)
memory usage: 1.2 GB


## installments_payments

In [98]:
# Preprocess installments_payments.csv
def installments_payments(num_rows=None, nan_as_category=True):
    ins = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\installments_payments.csv', nrows=num_rows)
    ins, cat_cols,ins_suppr_ = one_hot_encoder(ins, nan_as_category=nan_as_category)

    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)

    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    
    for cat in cat_cols:
        aggregations[cat] = ['mean']
        
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    
    # Compter les comptes d'installations
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    
    # Traitement des valeurs infinies
    ins_agg.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Imputation des valeurs manquantes
    ins_agg.fillna(ins_agg.mean(), inplace=True)

    # Identify columns that should not be scaled (categorical + SK_ID_CURR)
    encoded_columns = cat_cols  # Assuming cat_cols contains all categorical columns
    non_categorical_cols = ins_agg.columns.difference(encoded_columns).difference(['SK_ID_CURR'])  # Exclure SK_ID_CURR

    

    # Nettoyage de la mémoire
    del ins
    gc.collect()

    return ins_agg


In [99]:
df_installments=installments_payments(num_rows = None, nan_as_category = True)

In [100]:
df_installments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 339587 entries, 100001 to 456255
Data columns (total 26 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE  339587 non-null  int64  
 1   INSTAL_DPD_MAX                         339587 non-null  float64
 2   INSTAL_DPD_MEAN                        339587 non-null  float64
 3   INSTAL_DPD_SUM                         339587 non-null  float64
 4   INSTAL_DBD_MAX                         339587 non-null  float64
 5   INSTAL_DBD_MEAN                        339587 non-null  float64
 6   INSTAL_DBD_SUM                         339587 non-null  float64
 7   INSTAL_PAYMENT_PERC_MAX                339587 non-null  float64
 8   INSTAL_PAYMENT_PERC_MEAN               339587 non-null  float64
 9   INSTAL_PAYMENT_PERC_SUM                339587 non-null  float64
 10  INSTAL_PAYMENT_PERC_VAR                339587 non-null  

In [101]:
df_data = df_data.join(df_installments, how='left', on='SK_ID_CURR')

In [102]:
df_data.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,INSTAL_AMT_INSTALMENT_MEAN,INSTAL_AMT_INSTALMENT_SUM,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_MAX,INSTAL_AMT_PAYMENT_MEAN,INSTAL_AMT_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM,INSTAL_COUNT
0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,11559.247105,219625.695,9251.775,53093.745,11559.247105,219625.695,-49.0,-315.421053,-5993.0,19.0
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,64754.586000,1618864.650,6662.970,560835.360,64754.586000,1618864.650,-544.0,-1385.320000,-34633.0,25.0
2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,7096.155000,21288.465,5357.250,10573.965,7096.155000,21288.465,-727.0,-761.666667,-2285.0,3.0
3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,62947.088438,1007153.415,2482.920,691786.890,62947.088438,1007153.415,-12.0,-271.625000,-4346.0,16.0
4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,12666.444545,835985.340,0.180,22678.785,12214.060227,806127.975,-14.0,-1032.242424,-68128.0,66.0


In [103]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307507 entries, 0 to 307510
Columns: 669 entries, SK_ID_CURR to INSTAL_COUNT
dtypes: bool(146), float64(480), int64(43)
memory usage: 1.2 GB


## Credit_Card

In [104]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import gc

# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows=None, nan_as_category=True):
    cc = pd.read_csv('C:\\Users\\svenn\\Projet 7\\Projet+Mise+en+prod+-+home-credit-default-risk\\credit_card_balance.csv', nrows=num_rows)
    cc, cat_cols,cc_suppr = one_hot_encoder(cc, nan_as_category=True)
    
    # Drop SK_ID_PREV
    cc.drop(['SK_ID_PREV'], axis=1, inplace=True)

    # General aggregations
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])

    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()

    # Identify columns that should not be scaled (categorical + SK_ID_CURR)
    encoded_columns = cat_cols  # Assuming cat_cols contains all categorical columns
    non_categorical_cols = cc_agg.columns.difference(encoded_columns).difference(['SK_ID_CURR'])  # Exclure SK_ID_CURR
    

    

    del cc
    gc.collect()

    return cc_agg


In [105]:
df_credit_card=credit_card_balance(num_rows = None, nan_as_category = True)

In [106]:
df_credit_card.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103558 entries, 100006 to 456250
Columns: 141 entries, CC_MONTHS_BALANCE_MIN to CC_COUNT
dtypes: bool(16), float64(101), int64(24)
memory usage: 101.1 MB


In [107]:
df_data = df_data.join(df_credit_card, how='left', on='SK_ID_CURR')

In [108]:
df_data.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,6.0
4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307507 entries, 0 to 307510
Columns: 810 entries, SK_ID_CURR to CC_COUNT
dtypes: bool(146), float64(605), int64(43), object(16)
memory usage: 1.6+ GB


In [110]:
# Vérifier les valeurs infinies
infinity_mask = (df_data == np.inf) | (df_data == -np.inf)

# Afficher les colonnes contenant des valeurs infinies
infinite_columns = infinity_mask.any()

for col in df_data.columns[infinite_columns]:
    col_index = df_data.columns.get_loc(col)
    print(f"Colonne: {col}, Numéro dans le DataFrame initial: {col_index}")

# Créer df_data2 en supprimant les colonnes contenant des valeurs infinies
df_data2 = df_data.loc[:, ~infinite_columns]

Colonne: REFUSED_APP_CREDIT_PERC_MAX, Numéro dans le DataFrame initial: 605


In [111]:
df_data2.shape

(307507, 809)

In [112]:
# Reduction de l'échantillon pour gagner en vitesse de calcul pour train
# df_data2_sample = df_data2.sample(frac=0.10, random_state=42) # si je prend une partie
df_data2_sample=df_data2

In [113]:
df_data2_sample=df_data2

In [114]:
df_data2_sample.head()

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,False,0.0,0.0,0.0,False,False,0.0,0.0,0.0,6.0
4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
df_data2.shape

(307507, 809)

In [116]:
# Préparation sélection des donées X et Y

X = df_data2_sample.iloc[:, 2:]  # Toutes les colonnes sauf les deux premières
Y = df_data2_sample.iloc[:, 1]  # Colonne Tarket (variable cible)

In [117]:
# Répartition des données entraintement et test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=76, test_size=0.25)

In [118]:
X_train.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
270411,0,0,1,0,265500.0,1350000.0,67500.0,1350000.0,0.032561,-9353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109955,1,0,0,0,225000.0,1006920.0,70209.0,900000.0,0.006671,-18215,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165640,0,1,1,0,135000.0,808650.0,26217.0,675000.0,0.046220,-19326,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194469,1,0,0,0,148500.0,113746.5,10431.0,103500.0,0.007330,-23332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107940,0,0,0,1,135000.0,254700.0,14350.5,225000.0,0.046220,-16643,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
Y_train.head()

270411    0
109955    0
165640    0
194469    0
107940    0
Name: TARGET, dtype: int64

## réduction aux features sélectionnées

In [120]:
# Charger les caractéristiques sélectionnées depuis le fichier CSV

selected_features = pd.read_csv("selected_features.csv", header=None)

In [121]:
selected_features.head

<bound method NDFrame.head of              0           1              2            3                4   \
0  CNT_CHILDREN  DAYS_BIRTH  DAYS_EMPLOYED  OWN_CAR_AGE  FLAG_WORK_PHONE   

                5                            6             7             8   \
0  CNT_FAM_MEMBERS  REGION_RATING_CLIENT_W_CITY  EXT_SOURCE_1  EXT_SOURCE_2   

             9   ...                     15                              16  \
0  EXT_SOURCE_3  ...  PREV_CNT_PAYMENT_MEAN  PREV_NAME_CLIENT_TYPE_New_MEAN   

                             17                              18  \
0  APPROVED_APP_CREDIT_PERC_MIN  APPROVED_RATE_DOWN_PAYMENT_MIN   

                       19                                     20  \
0  POS_MONTHS_BALANCE_MAX  INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE   

               21                      22                      23  \
0  INSTAL_DPD_MAX  INSTAL_AMT_PAYMENT_MIN  INSTAL_AMT_PAYMENT_SUM   

                                24  
0  CC_AMT_DRAWINGS_ATM_CURRENT_SUM  

[1 rows x 25 

In [122]:
missing_columns = [col for col in selected_features if col not in X_train.columns]
if missing_columns:
    print(f"Colonnes manquantes : {missing_columns}")

Colonnes manquantes : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [123]:
type(X_train)

pandas.core.frame.DataFrame

In [124]:
print(selected_features)
print(type(selected_features))


             0           1              2            3                4   \
0  CNT_CHILDREN  DAYS_BIRTH  DAYS_EMPLOYED  OWN_CAR_AGE  FLAG_WORK_PHONE   

                5                            6             7             8   \
0  CNT_FAM_MEMBERS  REGION_RATING_CLIENT_W_CITY  EXT_SOURCE_1  EXT_SOURCE_2   

             9   ...                     15                              16  \
0  EXT_SOURCE_3  ...  PREV_CNT_PAYMENT_MEAN  PREV_NAME_CLIENT_TYPE_New_MEAN   

                             17                              18  \
0  APPROVED_APP_CREDIT_PERC_MIN  APPROVED_RATE_DOWN_PAYMENT_MIN   

                       19                                     20  \
0  POS_MONTHS_BALANCE_MAX  INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE   

               21                      22                      23  \
0  INSTAL_DPD_MAX  INSTAL_AMT_PAYMENT_MIN  INSTAL_AMT_PAYMENT_SUM   

                                24  
0  CC_AMT_DRAWINGS_ATM_CURRENT_SUM  

[1 rows x 25 columns]
<class 'pandas.core.f

In [125]:
selected_features = list(selected_features)

In [126]:
print(type(selected_features))

<class 'list'>


In [127]:
selected_features = X_train.columns[selected_features]

In [128]:
print(selected_features)

Index(['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL',
       'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START',
       'REG_REGION_NOT_LIVE_REGION'],
      dtype='object')


In [129]:
len(selected_features)

25

In [130]:
# Appliquer les caractéristiques à df_data2_sample
df_data2_sample_filtered = X_train[selected_features]

In [131]:
df_data2_sample_filtered.shape

(230630, 25)

In [132]:
# rajouter la colonne Target
df_data2_sample_filtered["TARGET"] = Y_train

In [133]:
df_data2_sample_filtered.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,TARGET
270411,0,0,1,0,265500.0,1350000.0,67500.0,1350000.0,0.032561,-9353,...,0,1,0,0,1.0,1,1,13,0,0
109955,1,0,0,0,225000.0,1006920.0,70209.0,900000.0,0.006671,-18215,...,0,1,0,0,2.0,2,2,16,0,0
165640,0,1,1,0,135000.0,808650.0,26217.0,675000.0,0.046220,-19326,...,0,1,0,0,2.0,1,1,16,0,0
194469,1,0,0,0,148500.0,113746.5,10431.0,103500.0,0.007330,-23332,...,0,1,1,0,1.0,2,2,17,0,0
107940,0,0,0,1,135000.0,254700.0,14350.5,225000.0,0.046220,-16643,...,0,1,1,0,3.0,1,1,12,0,0


In [134]:
df_data2_sample_filtered.shape

(230630, 26)

In [135]:
df_data2_sample_filtered.to_csv("df_data_train_non_scalées.csv", index=False)
print("Les données filtrées avec la colonne 'target' ont été sauvegardées dans 'df_data_train_non_scalées.csv.")

Les données filtrées avec la colonne 'target' ont été sauvegardées dans 'df_data_train_non_scalées.csv.
